# 데이터 시각화 1 - 지리데이터

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json

In [5]:
data_path = 'data/'
df1 = pd.read_csv(data_path + 'seoul_crime_by_office.csv',index_col=0)
df1.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별,lat,lng,검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구,37.563646,126.989580,1.275416
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구,37.575558,126.984867,1.523847
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구,37.554758,126.973498,0.907372
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구,37.564785,126.966776,1.978299
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구,37.571853,126.998914,1.198382


In [15]:
m = folium.Map(location=[37.5502, 126.982],zoom_start=11) # 맵 세팅

for i in range(len(df1)): # 마커 찍기
    folium.Marker([df1['lat'][i],df1['lng'][i]],
                             popup = (str(df1['관서명'][i])+str(df1['살인 발생'][i]))).add_to(m)

folium.CircleMarker(
  [37.552018,126.939577],
  radius=50,
  color='#ffffgg',
  fill_color='#fffggg',
  popup='Sogang University'
).add_to(m)    
m

In [10]:
# 마커 클러스터

from folium.plugins import MarkerCluster

m = folium.Map(location=[37.5502, 126.982],zoom_start=11)

marker_cluster = MarkerCluster().add_to(m)
for i in range(len(df1)): 
    folium.Marker([df1['lat'][i],df1['lng'][i]],
                             popup = (str(df1['관서명'][i])+str(df1['살인 발생'][i]))).add_to(marker_cluster)
    
m